In [1]:
import pandas as pd

In [2]:
events_train = pd.read_csv('https://stepik.org/media/attachments/course/4852/event_data_train.zip')
submissions_train = pd.read_csv('https://stepik.org/media/attachments/course/4852/submissions_data_train.zip')

Логически неверное решение, которое случайно даёт правильный результат

In [12]:
submissions_train_wrong = submissions_train[submissions_train['submission_status'] == 'wrong']

In [29]:
submissions_train_wrong.groupby('step_id').agg({'submission_status': 'count'}).sort_values('submission_status', ascending=False)

,submission_status
step_id,
31978,16084
32031,13373
32202,13002
33481,10300
32075,10003
...,...
33482,707
33681,703
33362,673


Верное решение с помощью merge (join в sql)

In [79]:
submissions_step_user_maxtime = submissions_train[submissions_train['submission_status'] == 'wrong']\
    .groupby(['step_id', 'user_id'], as_index=False)\
    .agg({'timestamp': 'max'})

In [81]:
submissions_user_lasttime = submissions_train.groupby('user_id', as_index=False).agg({'timestamp': 'max'})

In [93]:
submissions_merge = submissions_step_user_maxtime.merge(submissions_user_lasttime, on=['user_id', 'timestamp'])

In [99]:
submissions_merge.groupby('step_id').count().sort_values('user_id').tail(1)

,user_id,timestamp
step_id,,
31978,154,154


Верное решение из топа решений

In [96]:
submissions_train.sort_values(['user_id', 'timestamp'], ascending=False).drop_duplicates('user_id')\
    .query("submission_status == 'wrong'")\
    .groupby('step_id').count().sort_values('submission_status').tail(1)

,timestamp,submission_status,user_id
step_id,,,
31978,154,154,154
